In [2]:
# imports
import chess
import chess.engine as chess_engine
import chess.pgn as chess_pgn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io
import pathlib
from collections import Counter
#import utility
import tkinter as tk
from tkinter import ttk
from pathlib import Path
from IPython.display import Image as ima, display
import time
import os

In [3]:
path = pathlib.Path().resolve()
pgnfile = "./../games/game1.pgn"
enginepath = str(path)+"./../engines/stockfish/" 
enginefile = "stockfish-windows-x86-64-sse41-popcnt" 
totaltime = 2
with io.open(str(path) + pgnfile, encoding="utf-8-sig") as pgnin:
    game = chess_pgn.read_game(pgnin)

In [4]:
def material_balance(board):
    '''
    This function calculates the material 
    difference between White and Black.
    '''
    piece_values = {'p': 100, 'n': 300, 'b': 300, 'r': 500, 'q': 900, 'k': 20000}
    pieces = Counter(board.piece_map().values())
    white_material = sum(piece_values[p.symbol().lower()] for p in pieces if p.color == chess.WHITE)
    black_material = sum(piece_values[p.symbol().lower()] for p in pieces if p.color == chess.BLACK)
    return white_material - black_material

def calculate_development(board):
    '''
    This function evaluates how well each 
    side has developed their pieces.
    '''
    initial_squares = {
        chess.A1, chess.B1, chess.C1, chess.D1, chess.E1, chess.F1, chess.G1, chess.H1,
        chess.A8, chess.B8, chess.C8, chess.D8, chess.E8, chess.F8, chess.G8, chess.H8
    }
    development = 0
    for square in initial_squares:
        piece = board.piece_at(square)
        if piece:
            if (piece.color == chess.WHITE and square < chess.A2) or (piece.color == chess.BLACK and square > chess.H7):
                development -= 1
            else:
                development += 1
    return development

def calculate_mobility(board):
    '''
    This function calculates the number of legal 
    moves available to the current player.'''
    mobility = len(list(board.legal_moves))
    return mobility

def calculate_control(board):
    '''
    This function calculates the difference in 
    the number of squares controlled by each side.
    '''
    control = sum(len(board.attackers(chess.WHITE, square)) - len(board.attackers(chess.BLACK, square)) for square in chess.SQUARES)
    return control

def calculate_tension(board):
    '''
    This function calculates the number of squares 
    that are under attack by both White and Black.
    '''
    tension = sum(1 for square in chess.SQUARES if board.is_attacked_by(chess.WHITE, square) and board.is_attacked_by(chess.BLACK, square))
    return tension

def calculate_king_safety(board):
    '''
    calculate the number of pieces that are in 
    contact with the king's sensitive squares, by distance of rist
    '''
    safety = 0
    for king_square in [board.king(chess.WHITE), board.king(chess.BLACK)]:
        for piece_square in chess.SQUARES:
            piece = board.piece_at(piece_square)
            if piece:
                distance = max(chess.square_distance(king_square, piece_square), 1)
                #safety += piece.piece_type / distance
                safety += piece.piece_type
    return safety

In [ ]:
board = chess.Board()
gamedata = []
node = game
plytotal = sum(1 for _ in node.mainline())
time = totaltime * 80 / plytotal 
root = LoadingWindow("Game Analisys!")
counter = 0
moves_len = (len(str(game.mainline_moves()).split(".")) - 2) * 2
with chess.engine.SimpleEngine.popen_uci(enginepath + enginefile) as engine:
    node = game
    cap = 30  
    ply = 0
    matedist = "N/A"
    while not node.is_end():
        next_node = node.variations[0]
        move = node.board().san(next_node.move)
        side = "W" if board.turn else "B"
        capprior = cap

        try:
            result = engine.analyse(board, chess.engine.Limit(time=time))
            score = result['score'].relative

            if isinstance(score, chess.engine.Cp):
                cap = score.score()
            elif isinstance(score, chess.engine.Mate):
                cap = score.mate() * 10000 if score.mate() is not None else 0
                matedist = score.mate() if score.mate() is not None else "N/A"
            
            depth = result['depth']
            suggested = board.san(result['pv'][0]) if 'pv' in result else "N/A"

            if side == "B":
                cap = -cap

            cpdelta = cap - capprior

            board.push(next_node.move)  

            material = material_balance(board)
            development = calculate_development(board)
            mobility = calculate_mobility(board)
            control = calculate_control(board)
            tension = calculate_tension(board)
            safety = calculate_king_safety(board)

            movedata = (ply, side, move, cap, matedist, cpdelta, suggested, depth, material,
                        development, mobility, control, tension, safety)

            gamedata.append(movedata)

            ply += 1
            node = next_node
            counter += 1
            if counter == moves_len:
                root.close_window()
            root.update_progress(round((counter / moves_len * 100), 2))
            
            root.update()
        
        except Exception as e:
            print(f"Something wrong: {e}")
            break
gamedata = pd.DataFrame(gamedata, columns=['Ply', 'Side', 'Move', 'CP', 'Mate', 'CP Delta', 'Suggested', 'Depth', 'Material',
                                            'Development', 'Mobility', 'Control', 'Tension', 'Safety'])
gamedata['CP'] = gamedata['CP'].shift(-1)
gamedata['CP Delta'] = gamedata['CP Delta'].shift(-1)

In [6]:
class LoadingWindow(tk.Tk):
    def __init__(self, operation):
        super().__init__()
        self.title(operation)
        self.label = ttk.Label(text="Progress\n: 0%", font=("Arial", 15))
        self.info = ttk.Label(text="Progress\n: 0%", font=("Arial", 15))
        self.progressbar = ttk.Progressbar(orient=tk.HORIZONTAL, length=240, mode="determinate")
        self.button_close = ttk.Button(text="close", command=self.close_window)
        self.label.place(x=30, y=25)
        self.progressbar.place(x=30, y=50)
        self.button_close.place(x=20, y=5)
        self.progress = 0
        self.geometry("300x200")
    def update_progress(self, value):
        try:
            self.progress = value
            self.progressbar["value"] = self.progress
            self.label.config(text=f"Progress: {self.progress}%")
            if value == 100:
                time.sleep(2)
                self.destroy()
        except Exception as e:
            print(value,"**")

    def close_window(self):
        self.destroy()

    def update_info(self, value):
        self.label.config(text=value)

In [ ]:
inaccuracy = 100
mistake = 200
blunder=300
lost = 500
ok = 50
great = 25
excellent = 10
best = 1
moves = pd.DataFrame(columns=['Ply', 'Side', 'Move', 'CP', 'CP Delta', 'Type', 'Suggested', 'Depth'])
move_list = []
for index, row in gamedata.iterrows():
    move_type = None
    if row['Side'] == "B":
        if row['CP Delta'] > lost:
            move_type = 'Lost'
        elif row['CP Delta'] > blunder:
            move_type = 'Blunder'
        elif row['CP Delta'] > mistake:
            move_type = 'Mistake'
        elif row['CP Delta'] > inaccuracy:
            move_type = 'Inaccuracy'
        elif row['CP Delta'] <= best:
            move_type = 'Best Move'
        elif row['CP Delta'] <= excellent:
            move_type = 'Excellent'
        elif row['CP Delta'] <= great:
            move_type = 'Great'
        elif row['CP Delta'] <= ok:
            move_type = 'Good'
        else:
            move_type = 'Ok'
    else:
        if row['CP Delta'] < -lost:
            move_type = 'Lost'
        elif row['CP Delta'] < -blunder:
            move_type = 'Blunder'
        elif row['CP Delta'] < -mistake:
            move_type = 'Mistake'
        elif row['CP Delta'] < -inaccuracy:
            move_type = 'Inaccuracy'
        elif row['CP Delta'] >= -best:
            move_type = 'Best Move'
        elif row['CP Delta'] >= -excellent:
            move_type = 'Excellent'
        elif row['CP Delta'] >= -great:
            move_type = 'Great'
        elif row['CP Delta'] >= -ok:
            move_type = 'Good'
        else:
            move_type = 'Ok'
    if not move_type:
        move_type = "Lascou"
    move_list.append((row['Ply'], row['Side'], row['Move'], row['CP'], row['CP Delta'], move_type, row['Suggested'], row['Depth']))
moves = pd.DataFrame(move_list, columns=['Ply', 'Side', 'Move', 'CP', 'CP Delta', 'Type', 'Suggested', 'Depth'])
pd.options.display.max_rows = 1000
moves

In [11]:
grt = -2
ecl = -1
ok = -4
isc = -8
msc = -12
bsc = -24

whitegreat = 0
whiteok = 0
whiteexcellent = 0
whiteinaccuracies = 0
whitemistakes = 0
whiteblunders = 0

blackgreat = 0
blackok = 0
blackexcellent = 0
blackinaccuracies = 0
blackmistakes = 0
blackblunders = 0

totalwhite = len(gamedata[gamedata['Side']== "W"])*10
totalwhitescore = totalwhite
totalblack = len(gamedata[gamedata['Side']== "B"])*10
totalblackscore = totalblack

for index,row in moves.iterrows():
    if row['Side'] == "B":
        if row['Type'] == "Inaccuracy":
            totalblackscore += isc
            blackinaccuracies +=1
        elif row['Type'] == "Mistake":
            totalblackscore += msc
            blackmistakes +=1
        elif row['Type'] == "Blunder":
            totalblackscore += bsc
            blackblunders +=1
        elif row['Type'] == "Ok":
            totalblackscore += ok
            blackok +=1
        elif row['Type'] == "Great":
            totalblackscore += grt
            blackgreat +=1
        elif row['Type'] == "Excellent":
            totalblackscore += ecl
            blackexcellent +=1
      
    if row['Side'] == "W":
        if row['Type'] == "Inaccuracy":
            totalwhitescore += isc
            whiteinaccuracies +=1
        elif row['Type'] == "Mistake":
            totalwhitescore += msc
            whitemistakes +=1
        elif row['Type'] == "Blunder":
            totalwhitescore += bsc
            whiteblunders +=1
        elif row['Type'] == "Ok":
            totalwhitescore += ok
            whiteok +=1
        elif row['Type'] == "Great":
            totalwhitescore += grt
            whitegreat +=1
        elif row['Type'] == "Excellent":
            totalwhitescore += ecl
            whiteexcellent +=1
            
whitequality = float(totalwhitescore)/float(totalwhite)
blackquality = float(totalblackscore)/float(totalblack)            

print("Event:",game.headers["Event"])
print("Site:",game.headers["Site"])
print("Date:",game.headers["Date"])
print("Round:",game.headers["Round"])
print("White:",game.headers["White"])
print("Black:",game.headers["Black"])
print("Result:",game.headers["Result"])
print("="*60)
print("Quality of White Play:",round(whitequality*100,0),"%")
print("White made ",whiteok,"Ok moves,",whiteexcellent,"Excellent moves", whitegreat,"Great Moves")
print("White made",whiteinaccuracies,"inaccuracies,",whitemistakes,"mistakes, and", whiteblunders,"blunders.")
print("="*60)
print("Quality of Black Play:",round(blackquality*100,0),"%")
print("Black made ",blackok,"Ok moves,",blackexcellent,"Excellent moves", blackgreat,"Great Moves")
print("Black made",blackinaccuracies,"inaccuracies,",blackmistakes,"mistakes, and", blackblunders,"blunders.")

pgnin.close()

Event: Live Chess
Site: Chess.com
Date: 2024.07.20
Round: ?
White: DanteDietz
Black: FreeGaza19999
Result: 1-0
Quality of White Play: 71.0 %
White made  7 Ok moves, 3 Excellent moves 4 Great Moves
White made 2 inaccuracies, 0 mistakes, and 2 blunders.
Quality of Black Play: 66.0 %
Black made  6 Ok moves, 5 Excellent moves 7 Great Moves
Black made 3 inaccuracies, 0 mistakes, and 2 blunders.
